In [1]:
import pandas as pd
years = [18, 19, 20, 21, 22, 23, 24, 25]
headers = [
    "transaction_id",
    "price",
    "date_of_transfer",
    "postcode",
    "property_type",
    "new_build_flag",
    "tenure_type",
    "primary_addressable_object_name",
    "secondary_addressable_object_name",
    "street",
    "locality",
    "town_city",
    "district",
    "county",
    "ppd_category_type",
    "record_status"
]
drop = [
    "transaction_id",
    "locality",
    "ppd_category_type",
    "record_status"
]
dfs = {}
for year in years:
    url = f"http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-20{year}.csv"
    df = pd.read_csv(url)
    df.columns = headers
    df = df.drop(columns=drop)
    dfs[f"df20{year}"] = df
    print(f"{year} done")


KeyboardInterrupt: 

In [ ]:
df_full = pd.concat(dfs)

In [ ]:
df_full['Price (Thousands)'] = df_full['price'] / 1000
df_full.drop(columns=['price'], inplace=True)
df_full.describe()

In [ ]:
df_full.isna().sum()

In [ ]:
df_clean = df_full[df_full['postcode'].notna()]
df_clean.describe()

In [ ]:
df_binary = pd.get_dummies(df_clean , dtype=int, columns=['tenure_type','new_build_flag', 'property_type'])
df_binary.rename(columns={'date_of_transfer':'Transfer Date','town_city':'Town/City','district':'District','county':'County','tenure_type_F':'Freehold Tenure','tenure_type_L':'Leasehold Tenure','new_build_flag_N':'Old Build','new_build_flag_Y':'New Build','property_type_D':'Detached', 'property_type_F':'Flat', 'property_type_O':'Other Property Type','property_type_S':'Semi-detached','property_type_T':'Terraced', 'postcode':'Postcode'}, inplace=True)
df_binary.head()

In [ ]:
df_binary['Transfer Date'] = pd.to_datetime(df_binary['Transfer Date'], errors='coerce')
df_binary['Year'] = df_binary['Transfer Date'].dt.year
df_binary['Month'] = df_binary['Transfer Date'].dt.month
df_binary['Quarter'] = df_binary['Transfer Date'].dt.quarter
df_binary['Day of the Week'] = df_binary['Transfer Date'].dt.dayofweek
df_binary['Transfer Date'] = df_binary['Transfer Date'].astype(str).str[:10]

In [ ]:
print(df_binary.dtypes)

In [ ]:
print(df_binary.nunique())

In [ ]:
df_binary['Postcode Area'] = df_binary['Postcode'].str.extract(r'^([A-Z]+)')
df_binary['Postcode District'] = df_binary['Postcode'].str.extract(r'^([A-Z]+[0-9]+)')
df_binary.head()

In [ ]:
df_avg_pc = df_binary.groupby('Postcode Area')['Price (Thousands)'].mean().reset_index()
df_avg_pc.columns = ['Postcode Area', 'Area Code Average Price (Thousands)']
df_avg_county = df_binary.groupby('County')['Price (Thousands)'].mean().reset_index()
df_avg_county.columns = ['County', 'County Average Price (Thousands)']
df_avg_tc = df_binary.groupby('Town/City')['Price (Thousands)'].mean().reset_index()
df_avg_tc.columns = ['Town/City', 'Town/City Average Price (Thousands)']
df_avg_dist = df_binary.groupby('District')['Price (Thousands)'].mean().reset_index()
df_avg_dist.columns = ['District', 'District Average Price (Thousands)']
df_avg_pc.head()

In [ ]:
df_merge = df_binary.merge(df_avg_pc, on=['Postcode Area'], how='left')
df_merge = df_merge.merge(df_avg_tc, on=['Town/City'], how='left')
df_merge = df_merge.merge(df_avg_dist, on=['District'], how='left')
df_merge = df_merge.merge(df_avg_county, on=['County'], how='left')
df_merge.head()

In [ ]:
loc_df = pd.read_csv('ONSPD_Online_Latest_Centroids.csv')
loc_df.head()

In [ ]:
loc_df.columns

In [ ]:
loc_df_drop = loc_df[['LAT','LONG','PCDS','OSEAST1M','OSNRTH1M']]
loc_df_drop.rename(columns={'PCDS':'Postcode'}, inplace=True)
loc_df_drop.head()

In [ ]:
loc_house_df = df_merge.merge(loc_df_drop, on=['Postcode'], how='left')
loc_house_df.head()

In [ ]:
loc_house_df.isna().sum()

In [ ]:
print(loc_house_df['LONG'].min())
print(loc_house_df['LAT'].min())
print(loc_house_df['LONG'].max())
print(loc_house_df['LAT'].max())

In [ ]:
join_cols = [
    'Area Code Average Price (Thousands)',
    'Town/City Average Price (Thousands)',
    'District Average Price (Thousands)',
    'County Average Price (Thousands)',
    'LAT', 'LONG', 'OSEAST1M', 'OSNRTH1M',
    'Price (Thousands)', 'Freehold Tenure', 'Leasehold Tenure',
    'Old Build', 'New Build', 'Detached', 'Flat', 'Other Property Type',
    'Semi-detached', 'Terraced', 'Year', 'Month', 'Quarter',
    'Day of the Week'
]
loc_house_df = loc_house_df.drop_duplicates(subset=join_cols)

In [ ]:
upper = loc_house_df['Price (Thousands)'].quantile(0.95)
df_no_outliers =  loc_house_df[loc_house_df['Price (Thousands)'] <= upper]
df_no_outliers.head()

In [ ]:
df_no_outliers.to_csv('ETL_Final.csv', index=False)